# Settup

In [ ]:
!git clone https://github.com/hectorcarrion/ControllableGenDDI.git

Cloning into 'miccai-2025'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 73 (delta 40), reused 58 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (73/73), 177.78 KiB | 8.89 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [ ]:
hf_token = # replace with your github token
!huggingface-cli login --token {hf_token}

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `colab` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `colab`


# Train LoRA

In [ ]:
!pip install bitsandbytes diffusers==0.22.0 huggingface_hub==0.25.0
!pip uninstall -y peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import os
import shutil
import glob
import torch
from safetensors import safe_open
from safetensors.torch import save_file

In [ ]:
SEED = 0
SPLIT = f"ddi_{SEED}"

OUTPUT_DIR_BASE = f".."
path = f"{OUTPUT_DIR_BASE}/*/learned_embeds.safetensors"
target_path = f"{OUTPUT_DIR_BASE}/aggregated_embeds_SEED={SEED}.pt"

In [ ]:
merged_dict = dict()
for file in glob.glob(path):
    tensors = {}
    with safe_open(file, framework="pt", device="cpu") as f:
        for key in f.keys():
            tensors[key] = f.get_tensor(key)
        merged_dict.update(tensors)

os.makedirs(os.path.dirname(target_path), exist_ok=True)
torch.save(merged_dict, target_path)
print(merged_dict)

{'<abscess-class>': tensor([[ 0.0336, -0.0929,  0.0927,  ...,  0.1356, -0.1197, -0.0410]]), '<acne-cystic-class>': tensor([[ 0.0144, -0.0685,  0.0166,  ...,  0.0737,  0.0434, -0.0136]]), '<acquired_digital_fibrokeratoma-class>': tensor([[ 0.0837,  0.1314, -0.1585,  ...,  0.0369, -0.0451,  0.2210]]), '<acral_melanotic_macule-class>': tensor([[-0.0022,  0.0303, -0.0297,  ...,  0.0624,  0.1172, -0.1356]]), '<acrochordon-class>': tensor([[ 0.0146, -0.0569, -0.1451,  ..., -0.0315,  0.2035,  0.0306]]), '<actinic_keratosis-class>': tensor([[-0.0003, -0.0835,  0.0214,  ...,  0.0085,  0.1182,  0.0260]]), '<angioleiomyoma-class>': tensor([[-0.0522, -0.0379, -0.0166,  ...,  0.0870, -0.0146,  0.1065]]), '<angioma-class>': tensor([[-0.1210,  0.0884,  0.0724,  ..., -0.0136,  0.2079,  0.0167]]), '<arteriovenous_hemangioma-class>': tensor([[-0.1179, -0.1670, -0.0120,  ...,  0.0979,  0.1182, -0.1374]]), '<basal_cell_carcinoma-class>': tensor([[-0.0282, -0.1182,  0.0768,  ...,  0.0836,  0.0495,  0.0845]

In [ ]:
import json
import os

### if you want to use images of light skin only for training, comment out the last two parts in the concept list.
def make_concepts_list(special_token, disease):
    # instace_data_dir: specificaly the data for the disease (and I assume skin color)
    # class_data_dir: the data for the skin color (anything else)?
    base_instance = f'./' # the repo with your instance data (diseased data, DDI)
    base_class = f'./' # the repo with your class data (healthy synthetics)
    concepts_list = [
        {
            "instance_prompt":      f"An image of {special_token} on the skin of a light-skinned individual",
            "instance_data_dir":    f"{base_instance}/light/{disease}", # 1 is Fitzpatrick scale for skin-tone
            "class_prompt":         "An image of healthy skin on a light-skinned individual", # insert marker?
            "class_data_dir":       f"{base_class}/light/",
            "label":                0,
        },
        {
            "instance_prompt":      f"An image of {special_token} on the skin of a medium-skinned individual",
            "instance_data_dir":    f"{base_instance}/medium/{disease}", # 2 is Fitzpatrick scale for skin-tone
            "class_prompt":         "An image of healthy skin on a medium-skinned individual", # insert ruler?
            "class_data_dir":       f"{base_class}/medium/",
            "label":                1,
        },
        {
            "instance_prompt":      f"An image of {special_token} on the skin of a dark-skinned individual",
            "instance_data_dir":    f"{base_instance}/dark/{disease}", # 5 is Fitz
            "class_prompt":         "An image of healthy skin on a dark-skinned individual", # only ruler is easy
            "class_data_dir":       f"{base_class}/dark/", # where that data lives?
            "label":                2,

        },
    ]

    for c in concepts_list:
        os.makedirs(c["instance_data_dir"], exist_ok=True) # creates all above jsons

    with open(f"ti_lora_concepts_list_seed={SPLIT}_{SEED}.json", "w") as f:
        json.dump(concepts_list, f, indent=4)

In [ ]:
diseases = [
    "abscess", "acne-cystic", "acquired digital fibrokeratoma", "acral melanotic macule",
    "acrochordon", "actinic keratosis", "angioleiomyoma", "angioma",
    "arteriovenous hemangioma", "basal cell carcinoma", "benign keratosis",
    "blastic plasmacytoid dendritic cell neoplasm", "blue nevus", "cellular neurothekeoma",
    "chondroid syringoma", "clear cell acanthoma", "coccidioidomycosis",
    "condyloma acuminatum", "cutaneous T-cell lymphoma", "dermatofibroma",
    "dermatomyositis", "eccrine poroma", "eczema/atopic dermatitis",
    "epidermal cyst", "epidermal nevus", "fibrous papule", "focal-acral-hyperkeratosis",
    "folliculitis", "foreign body granuloma", "glomangioma", "graft-vs-host disease",
    "hematoma", "hyperpigmentation", "kaposi sarcoma", "keloid", "leukemia cutis",
    "lichenoid keratosis", "lipoma", "lymphocytic infiltrations", "melanoma",
    "metastatic carcinoma", "molluscum contagiosum", "neurofibroma", "neuroma", "nevus",
    "nevus lipomatosus superficialis", "onychomycosis", "prurigo nodularis",
    "pyogenic granuloma", "reactive lymphoid hyperplasia", "scar", "scleroderma/morphea",
    "sebaceous carcinoma", "seborrheic keratosis", "solar lentigo", "spindle cell nevus of Reed",
    "squamous cell carcinoma", "syringocystadenoma papilliferum", "tinea pedis",
    "traumatic injury", "trichilemmoma", "trichofolliculoma", "verruca vulgaris",
    "verruciform xanthoma", "xanthogranuloma"] # All diseases in DDI

In [ ]:
token_to_disease = {}
for disease in diseases:
    # Replace spaces with underscores
    safe_disease = disease.replace(" ", "_")
    # Construct the token
    token = f"<{safe_disease}-class>"
    # Populate the dictionary
    token_to_disease[token] = disease

In [ ]:
token_to_disease

{'<abscess-class>': 'abscess',
 '<acne-cystic-class>': 'acne-cystic',
 '<acquired_digital_fibrokeratoma-class>': 'acquired digital fibrokeratoma',
 '<acral_melanotic_macule-class>': 'acral melanotic macule',
 '<acrochordon-class>': 'acrochordon',
 '<actinic_keratosis-class>': 'actinic keratosis',
 '<angioleiomyoma-class>': 'angioleiomyoma',
 '<angioma-class>': 'angioma',
 '<arteriovenous_hemangioma-class>': 'arteriovenous hemangioma',
 '<basal_cell_carcinoma-class>': 'basal cell carcinoma',
 '<benign_keratosis-class>': 'benign keratosis',
 '<blastic_plasmacytoid_dendritic_cell_neoplasm-class>': 'blastic plasmacytoid dendritic cell neoplasm',
 '<blue_nevus-class>': 'blue nevus',
 '<cellular_neurothekeoma-class>': 'cellular neurothekeoma',
 '<chondroid_syringoma-class>': 'chondroid syringoma',
 '<clear_cell_acanthoma-class>': 'clear cell acanthoma',
 '<coccidioidomycosis-class>': 'coccidioidomycosis',
 '<condyloma_acuminatum-class>': 'condyloma acuminatum',
 '<cutaneous_T-cell_lymphoma-c

In [ ]:
skin_types = {
    'light': 'a light-skinned',
    'medium': 'a medium-skinned',
    'dark': 'a dark-skinned',
}

for token, disease in token_to_disease.items():
    print(token, disease)
    make_concepts_list(token, disease)
    concept_list = f"ti_lora_concepts_list_seed={SPLIT}_{SEED}.json"

    MODEL_NAME = "stabilityai/stable-diffusion-2-1-base"
    OUTPUT_DIR = f"./"
    embed_path = target_path

    print(f"[*] Weights will be saved at {OUTPUT_DIR}")
    !accelerate launch train_lora.py \
        --pretrained_model_name_or_path=$MODEL_NAME \
        --output_dir="$OUTPUT_DIR" \
        --revision="fp16" \
        --seed=$SEED \
        --resolution=256 \
        --train_batch_size=16 \
        --with_prior_preservation \
        --num_class_images=96 \
        --sample_batch_size=16 \
        --mixed_precision="fp16" \
        --use_8bit_adam \
        --gradient_accumulation_steps=1 \
        --gradient_checkpointing \
        --learning_rate=5e-6 \
        --lr_scheduler="constant" \
        --lr_warmup_steps=0 \
        --max_train_steps=750 \
        --checkpointing_steps=250 \
        --concepts_list=$concept_list \
        --rank=8 \
        --embed_path="$embed_path"

<abscess-class> abscess
[*] Weights will be saved at /content/drive/MyDrive/ucsc_drive/UCSC/Winter 2025/lora/ddi_2/ti_lora_SEED=0/abscess
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2025-02-22 21:09:52.002022: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740258592.021856    4197 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740258592.027913    4197 cuda_blas.cc:1418] Unable to register cuBLAS factory: At